Postprocess for HMM

In [1]:
import os
import re
import numpy as np
import pronouncing
import random

In [2]:
syllable_dict = open(os.path.join(os.getcwd(), 'Syllable_dictionary.txt')).read()

In [3]:
text = open(os.path.join(os.getcwd(), 'shakespeare_rhyme.txt')).read()
lines = [line for line in text.split('\n')]
all_poems = []
new_lines = []
for line in lines:
    if line and bool(re.search(r'\d', line)):
        all_poems.append(new_lines)
        new_lines = []
    else:
        if line:
            new_lines.append(line)
all_poems.append(new_lines)
all_poems = all_poems[1:]

In [4]:
rhyme_words = []
punc = [',', '.', '?', ':', '!', ';', "'", ')']
for poem in all_poems:
    for i in range(3):
        for j in range(2):
            line1 = poem[4*i + j].split(' ')
            line2 = poem[4*i + 2 + j].split(' ')
            word1 = line1[-1]
            word2 = line2[-1]
            if word1[-1] in punc:
                word1 = word1[0:-1]
            if word2[-1] in punc:      
                word2 = word2[0:-1]
            rhyme_words.append((word1, word2))
    line1 = poem[12].split(' ')
    line2 = poem[13].split(' ')
    word1 = line1[-1]
    word2 = line2[-1]
    if word1[-1] in punc:
        word1 = word1[0:-1]
    if word2[-1] in punc:      
        word2 = word2[0:-1]
    rhyme_words.append((word1, word2))

In [5]:
syllable_table = {}
lines = [line for line in syllable_dict.split('\n')]
for line in lines:
    line = line.split(' ')
    if 'E' in line[1]:
        syllable_table[line[0]] = int(line[2])
    else:
        syllable_table[line[0]] = int(line[1])

In [6]:
hmm_results = open(os.path.join(os.getcwd(), 'raw_result_hmm_16.txt')).read()

In [7]:
print(hmm_results)

Come servant's and breed gives and what , i converted the treasure , drawn lovers true lame to chide to thy love despise so boast , even buriest thy too , than pipe to one found love absent than mind chase of thine . for seemed more gone not , as thus in junes birth give palate this face ? nor thrivers see end injurious , is have sweet in ay , be think gold in me . as not thou , and i my be you , and thou my good due of dedicated fair mayst thee night and tables , me the so the a have in foot , what need gift ? she : beauty love roses i i i gentle thy all a though them forfeit , kind , thou best book outlive , nor on by taker thou brief to and self the love . and heart might how thus that make accumulate too all have trust , but deceased to votary ! brief ? what that least ? and beauty , that prognosticate am a loss receives bad , this time , keeps the year well pine , to the truth extreme ? my this ? those did vanishing , that men their foot siege in immortal keep you spur afterwards 

In [8]:
words = [word for word in hmm_results.split(' ')]

In [9]:
punc = [',', '.', '?', ':', '!', ';']
useless = ['(', ')', "'"]
poem = []
sentence = []
syllable_count = 0
for word in words:
    if word in useless:
        continue
    if word == 'i':
        word = 'I'
    if word not in punc:
        sentence.append(word)
        if word in syllable_table:
            s = syllable_table[word]
        else:
            s = 1
        syllable_count += s
    else:
        sentence.append(word)
        if syllable_count > 9 and syllable_count < 12:
            poem.append(sentence)
        sentence = []
        syllable_count = 0

final_results = []
for sentence in poem:
    line = ''
    for word in sentence:
        line += word
        line += ' '
    line = line[0:-3] + line[-2]
    line = line[0].upper() + line[1:]
    print(line)
    final_results.append(line)

As thus in junes birth give palate this face?
That prognosticate am a loss receives bad,
And which grievances on thine you prevailed,
Now thy gilding counterfeit shade will and I,
She being on lip is with expense all wonder:
Bear thee despised doth to his love's age,
Thee moon mistaking worthy tyranny.
Quenched her physician whose beauty's gems,
As where comes false wand'rest on most I essays.
Which the the ne'er my spirit falsehood imprint lay,
Whereon happy eye means when whose thy fortune,
And thus those with posterity of you,
When for as fair beloved for my report,
So thy parts so ruth falsehood now much reeks,
In she beauty prove but face doth at your muse,
And hides have him bear have bear scarlet weeds,
Fairer of the charged my the down to my fresh,
He of time's sway with ah hold it wise seek,
What incapable I I call never,
Which the scandal of worst doth sullied in face,
Then that must love weary even to and the heart,
Afresh and all is both of his familiar,
Though on a to from

Believed the heaven kindness growing in my spend,
Neither thou this arising are book are thee,
Those keen are breath matter let thy the truth,
Could by stay dost the knowing my time new:
For thou not thou art eyes should of your service,
Then with body's may t' where to I in I.
That denied so shall she within for taken;
Object time time I aught I I hath well souls,
Were as with soul's sight of beauty's subtleties,
Mine blood and o wrinkle a sinful of sight,
Most reeks man outward acquainted removed truth,
And this that new is if can me eye's a,
And more must in the eye huge purpose tongues you,
Yet I had call dull those man beauteous my love,
But in love's uphold painted advantage,
Despite not not come torture wood should all own,
Which contains to when to affable thought,
Famished age a wherein but perjured to thy,
Coral but the and all upon their behind,
Cast that female aid from my partly youth,
When art on stand to boldness unless is,
I not not looks hath whilst better by they thos

In [18]:
M = len(poem)
part1 = []
part2 = []
used = []
while True:
    if len(part1) >= 7 and len(part2) >= 7:
        break
    [i, j] = random.sample(range(M),  2)
    sentence1 = poem[i]
    sentence2 = poem[j]
    lastword1 = sentence1[-2]
    lastword2 = sentence2[-2]
    if i in used or j in used or lastword1 == lastword2:
        continue
    if (lastword1, lastword2) in rhyme_words or (lastword2, lastword1) in rhyme_words or lastword1 in pronouncing.rhymes(lastword2):
        used.append(i)
        used.append(j)
        part1.append(sentence1)
        part2.append(sentence2)

final_poem = []
for i in range(3):
    final_poem.append(part1[2*i])
    final_poem.append(part1[2*i + 1])
    final_poem.append(part2[2*i])
    final_poem.append(part2[2*i + 1])
final_poem.append(part1[6])
final_poem.append(part2[6])


final_product = []
for sentence in final_poem:
    line = ''
    for word in sentence:
        line += word
        line += ' '
    line = line[0:-3] + line[-2]
    line = line[0].upper() + line[1:]
    final_product.append(line)

for i in range(3):
    print(final_product[4*i])
    print(final_product[4*i + 1])
    print(final_product[4*i + 2])
    print(final_product[4*i + 3])
    print('\n')
print(final_product[12])
print(final_product[13])

Good me is the unfathered to away.
Let so argument so my thy light's though soon,
How course on as tender thy is still they,
Worship but thou sweet a not lose and moon.


Counting long thy lie for your self high fair:
Sourest would love if what against doth pitch thee,
My any view of ride cause at outward heir,
Exceeded use since life my dead can gain be,


Thief thee of the I care hath thee glass again,
And of thy on review glass kind from decay,
And else could seest the ground ink own remain?
But record weed the defence untutored day,


That sharp'st expense powers side wrack each good you,
Whilst their sweet nay date touched play your my due,
